In [2]:
import pandas as pd
m_df = pd.read_csv('/home/ndo/bash_output/variants_df/variants_df.csv', sep=",",encoding='utf-8')

In [3]:
m_df

,CHROM,POS,REF_GIAB,ALT_GIAB,REF_S,ALT_S,VAR_CATE
0,chr1,783175,T,C,NaN,NaN,FN
1,chr1,784860,T,C,NaN,NaN,FN
2,chr1,785417,G,A,NaN,NaN,FN
3,chr1,797392,G,A,NaN,NaN,FN
4,chr1,798618,C,T,NaN,NaN,FN
...,...,...,...,...,...,...,...
3667132,chr9,138216697,C,G,NaN,NaN,FN
3667133,chr9,138217446,C,T,NaN,NaN,FN
3667134,chr9,138217868,T,C,NaN,NaN,FN
3667135,chr9,138219722,A,G,NaN,NaN,FN


vcf file has 1-based position, purpose is to convert this vcf pos to bed files of left and right flanking sequence (0-based).
from the bed files extract the flanking sequences using bedtools getfasta

In [19]:
df = m_df[['CHROM', 'POS']]
df

,CHROM,POS
0,chr1,783175
1,chr1,784860
2,chr1,785417
3,chr1,797392
4,chr1,798618
...,...,...
3667132,chr9,138216697
3667133,chr9,138217446
3667134,chr9,138217868
3667135,chr9,138219722


In [20]:
#GET THE LEFT FLANKING SEQUENCE

left_df = df.copy(deep=True)
left_df['START'] = left_df['POS'] - 101 #start is incluive
left_df['END'] = left_df['POS']
left_df.drop(['POS'], axis=1, inplace=True)
left_df

,CHROM,START,END
0,chr1,783074,783175
1,chr1,784759,784860
2,chr1,785316,785417
3,chr1,797291,797392
4,chr1,798517,798618
...,...,...,...
3667132,chr9,138216596,138216697
3667133,chr9,138217345,138217446
3667134,chr9,138217767,138217868
3667135,chr9,138219621,138219722


In [22]:
right_df = df.copy(deep=True)
right_df['START'] = right_df['POS'] #start is incluive - 1 for bed file and  +1 for next nucleotide
right_df['END'] = right_df['POS'] + 100
right_df.drop(['POS'], axis=1, inplace=True)

right_df

,CHROM,START,END
0,chr1,783175,783275
1,chr1,784860,784960
2,chr1,785417,785517
3,chr1,797392,797492
4,chr1,798618,798718
...,...,...,...
3667132,chr9,138216697,138216797
3667133,chr9,138217446,138217546
3667134,chr9,138217868,138217968
3667135,chr9,138219722,138219822


In [23]:
left_df.to_csv('/home/ndo/bash_output/left_flanking/left_flanking.bed', sep='\t', index=False, header=False)


In [24]:
%%bash

bedtools getfasta -tab \
    -fo /home/ndo/bash_output/left_flanking/left_flanking.txt  \
    -fi /home/ndo/GIABv3Ref/GRCh38_GIABv3_no_alt_analysis_set_maskedGRC_decoys_MAP2K3_KMT2C_KCNJ18.fasta \
    -bed /home/ndo/bash_output/left_flanking/left_flanking.bed